### 📖 Stuff Documents 체인을 사용하여 완전한 RAG 파이프라인을 구현

- 체인을 `수동으로 구현`해야 합니다.

- 체인에 `ConversationBufferMemory` 를 부여합니다.

- 해당 문서를 사용하여 RAG를 수행합니다 : https://gist.github.com/serranoarevalo/5acf755c2b8d83f1707ef266b82ea223

- 체인에 다음 질문을 합니다:
    - Aaronson 은 유죄인가요?
    - 그가 테이블에 어떤 메시지를 썼나요?
    - Julia 는 누구인가요?

#### 💡 사용할 라이브러리 및 모듈 import 하기

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder 

from langchain.storage import LocalFileStore
from langchain.embeddings import CacheBackedEmbeddings
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

#### 💡 언어모델(LLM) 생성하기

In [2]:
llm = ChatOpenAI(
    temperature=0.1,        # 창의성 (0 ~ 2)
    model='gpt-3.5-turbo',  # 사용 모델 지정 (Default : gpt-3.5-turbo)
)

#### 💡 메모리 생성 

In [3]:
memory = ConversationBufferMemory(
    max_token_limit=120,
    return_messages=True,           # 문자열 기반이 아닌, ChatPromptTemplate 에서 사용할 수 있는 형태로 반환
)

#### 💡 1단계 : `문서 로드 (Load Documents)`

In [4]:
loader = UnstructuredFileLoader("./files/challenge.txt")

#### 💡 2단계 : `텍스트 분할 (Split Text)`

In [5]:
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",                 # 특정 기준으로 분할
    chunk_size=600,
    chunk_overlap=100,
)

split_doc = loader.load_and_split(text_splitter)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\tkdgu\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\tkdgu\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


#### 💡 3단계 : `임베딩 (Embedding) 생성`

In [6]:
embeddings = OpenAIEmbeddings()

# 캐시 지정
cache_dir = LocalFileStore('./.cache/')

# 캐시를 활용한 임베딩
cacehd_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir
)

#### 💡 4단계 : `DB 생성 및 저장`

In [7]:
vectorstore = FAISS.from_documents(documents=split_doc, embedding=cacehd_embeddings)

#### 💡 5단계 : `검색기(Retriever) 생성`

In [8]:
retriever = vectorstore.as_retriever()

#### 💡 6단계 : `프롬프트(prompt) 생성`

In [9]:
prompt = ChatPromptTemplate.from_messages([
    ("system", 
    """
    You are an assistant for question-answering tasks. 
    Use the following pieces of retrieved {context} to answer the question. 
    If you don't know the answer, just say that you don't know. 
    Answer in Korean.
    """
    ),
    # MessagesPlaceholder(variable_name='history'),
    ("human", "{question}")
])

#### 💡 7단계 : `Stuff Documenet Chain 안에 Memory를 연결한 Class 정의`

In [10]:
class StuffDocumentMemoryChain:
    def __init__(self, llm, prompt, memory, retriver, input_key="question"):
        self.prompt = prompt
        self.memory = memory
        self.retriver = retriver
        self.input_key = input_key
        self.chain = {
            'context': retriver, 
            'history': self.load_memory,
            'question': RunnablePassthrough(),
        } | prompt | llm

    def load_memory(self, _):
        return self.memory.load_memory_variables({})['history']

    def invoke(self, question):
        answer = self.chain.invoke(question)
        self.memory.save_context({"input": question}, {"output": answer.content})
        
        return answer.content

#### 💡 8단계 : `체인(Chain) 생성`

In [11]:
stuff_document_memory_chain = StuffDocumentMemoryChain(llm, prompt, memory, retriever)

#### 📢 [ 질문 1 ] Aaronson 은 유죄인가요 ?

In [12]:

# [ 질문 1 ] Aaronson 은 유죄인가요 ?
stuff_document_memory_chain.invoke('Is Aaronson guilty?')

'제가 알기로는, Aaronson은 유죄로 판결받았습니다.'

#### 📢 [ 질문 2 ] 그가 테이블에 어떤 메시지를 썼나요?

In [13]:

# [ 질문 2 ] 그가 테이블에 어떤 메시지를 썼나요?
stuff_document_memory_chain.invoke('What message did he write on the table?')

'그는 테이블 위에 손가락으로 먼지에 흔적을 남겼고 "2+2=5"라고 쓴 것을 추적했습니다.'

#### 📢 [ 질문 3 ] Julia 는 누구인가요?

In [14]:

# [ 질문 3 ] Julia 는 누구인가요?
stuff_document_memory_chain.invoke('Who is Julia?')

'제시된 문서에서 Julia는 윈스턴과 관련된 인물로서, 그의 사랑이자 동료입니다. 그녀는 윈스턴과 함께 Party에 저항하고자 하는 의지를 나타내며, 윈스턴이 자신을 버리고 그녀를 대신해서 고통을 받기를 바라는 마음을 품게 합니다. Julia는 윈스턴에게 중요한 인물로서, 그의 행동과 마음을 변화시키는데 영향을 미치는 역할을 합니다.'

#### 📢 메모리를 출력하여 메모리가 체인에 적용되었는지 확인

In [15]:
stuff_document_memory_chain.load_memory({})

[HumanMessage(content='Is Aaronson guilty?'),
 AIMessage(content='제가 알기로는, Aaronson은 유죄로 판결받았습니다.'),
 HumanMessage(content='What message did he write on the table?'),
 AIMessage(content='그는 테이블 위에 손가락으로 먼지에 흔적을 남겼고 "2+2=5"라고 쓴 것을 추적했습니다.'),
 HumanMessage(content='Who is Julia?'),
 AIMessage(content='제시된 문서에서 Julia는 윈스턴과 관련된 인물로서, 그의 사랑이자 동료입니다. 그녀는 윈스턴과 함께 Party에 저항하고자 하는 의지를 나타내며, 윈스턴이 자신을 버리고 그녀를 대신해서 고통을 받기를 바라는 마음을 품게 합니다. Julia는 윈스턴에게 중요한 인물로서, 그의 행동과 마음을 변화시키는데 영향을 미치는 역할을 합니다.')]